In [14]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


import numpy as np
import pandas as pd

In [15]:
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data,
                    columns=boston_dataset.feature_names)
# data.head()
features = data.drop(['INDUS','AGE'], axis=1)
# features.head()
log_prices = np.log(boston_dataset.target)
# log_prices.shape
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [16]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8
# property_stats = np.ndarray(shape = (1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)

In [17]:
type(features.mean().values)

numpy.ndarray

In [18]:
features.mean().values.reshape(1, 11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [19]:
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [20]:
regr = RandomForestRegressor().fit(features, target)
# regr = LinearRegression().fit(features, target)

fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)
RMSE

<ipython-input-20-a312f40d9530>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr = RandomForestRegressor().fit(features, target)


0.05702371279951113

In [21]:
def get_log_estimate(nr_rooms,
                     students_per_classroom,
                     next_to_river=False,
                     high_confidence=True):
    
    # Confidence property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom

    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    
    # Make prediction
    log_estimate = regr.predict(property_stats)
    
    # Clac Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68


    return log_estimate, upper_bound, lower_bound, interval

In [22]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(array([2.94080906]), array([2.99783278]), array([2.88378535]), 68)

In [23]:
np.median(boston_dataset.target)

21.2

In [28]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

# SCALE_FACTOR
log_est, upper, lower, conf = get_log_estimate(nr_rooms=3, 
                                    students_per_classroom=4,
                                    next_to_river=True,
                                    high_confidence=False)
#Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_high = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Rounded the dollar values to nearest thousand
rounded_est = np.round(dollar_est, 2)
rounded_high = np.round(dollar_high, 2)
rounded_low = np.round(dollar_low, 2)

# print(rounded_est, rounded_high, rounded_low)
print(f'The estimated property value is USD {rounded_est[0]}')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low[0]} at the lower end to USD {rounded_high[0]} at the high end.')

The estimated property value is USD 525359.93
At 68% confidence the valuation range is
USD 496240.11 at the lower end to USD 556188.53 at the high end.


In [30]:
def get_dollar_estimate(rm,pt_ratio,next_to_river=False,large_range=True):
    """Estimate the price of a property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    pt_ratio -- number of studentd per teacher in the classroom for the school in the area
    next_to_river -- True if the property is next to river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    """
    if rm < 1 or pt_ratio < 1:
        print('That is unrealistic, try again')
        return     
    
    log_est, upper, lower, conf = get_log_estimate(nr_rooms=3, 
                                        students_per_classroom=pt_ratio,
                                        next_to_river=next_to_river,
                                        high_confidence=large_range)
    #Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_high = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Rounded the dollar values to nearest thousand
    rounded_est = np.round(dollar_est, 2)
    rounded_high = np.round(dollar_high, 2)
    rounded_low = np.round(dollar_low, 2)

    print(f'The estimated property value is USD {rounded_est[0]}')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low[0]} at the lower end to USD {rounded_high[0]} at the high end.')    

In [31]:
get_dollar_estimate(rm=10, pt_ratio=20, next_to_river=False)

The estimated property value is USD 521294.21
At 95% confidence the valuation range is
USD 465106.84 at the lower end to USD 584269.3 at the high end.


In [32]:
import boston_valuation as val

val.get_dollar_estimate(6,12, True)

(506, 1)
The estimated property value is USD 596372.73
At 95% confidence the valuation range is
USD 409870.64 at the lower end to USD 867738.26 at the high end.


In [33]:
import pickle
with open('boston_house_prices_rf_model.pickle', 'wb') as f:
    pickle.dump(regr, f)

In [19]:
import json
columns = {
    'data_columns' : [col.lower() for col in features.columns]
}
with open('boston_house_prices_preds.json', 'w') as f:
    f.write(json.dumps(columns))